In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm


def read_and_prepare_data(base_path, rmt_path):
	"""
	读取所有个股数据，计算收益率 rit，合并市场收益率 rmt，准备成标准格式
	返回包含：['DATE', 'CODE', 'CLOSE', 'RET'] 的标准面板数据
	"""
	df_all_trd = pd.DataFrame()
	files = os.listdir(base_path)[:-2]
	rmt_df = pd.read_excel(rmt_path)
	rmt_df.rename(columns={'date': 'Trddt'}, inplace=True)
	rmt_df['Trddt'] = rmt_df['Trddt'].astype(str)

	# 添加外层进度条：年文件夹
	for file in tqdm(files[7:33], desc="年份进度"):
		package2 = os.path.join(base_path, file)
		files2 = os.listdir(package2)

		# 添加内层进度条：每年中的个股文件
		for file2 in tqdm(files2, desc=f"{file} 股票文件", leave=False):
			df_year = pd.read_excel(os.path.join(package2, file2))
			df_all_trd = pd.concat([df_all_trd, df_year])

	df_all_trd.drop(index=(df_all_trd.loc[(df_all_trd['Scode'] == '股票代码')].index), inplace=True)
	df_all_trd['Stkcd'] = 'a' + df_all_trd['Scode']
	df_all_trd['Dclsprc'] = df_all_trd['Dclsprc'].astype(float)
	df_all_trd['Dclsprcp'] = df_all_trd['Dclsprcp'].astype(float)
	df_all_trd['rit'] = (df_all_trd['Dclsprc'] - df_all_trd['Dclsprcp']) / df_all_trd['Dclsprcp']

	df_all_trd = pd.merge(df_all_trd, rmt_df[['Trddt', 'rmt']], on='Trddt', how='left')
	df_all_trd.rename(columns={'Trddt': 'DATE', 'Stkcd': 'CODE', 'Dclsprc': 'CLOSE'}, inplace=True)
	df_all_trd['DATE'] = pd.to_datetime(df_all_trd['DATE'])
	df_all_trd['RET'] = df_all_trd['rit']

	return df_all_trd[['DATE', 'CODE', 'CLOSE', 'RET']].dropna()

In [ ]:
# 设置路径
base_path = r'D:\data\个股日回报率\\'
rmt_path = r'D:\data\expanded_model_return.xlsx'

# 读取数据
df_panel = read_and_prepare_data(base_path, rmt_path)